## Train test split for targetlist

In [44]:
import os
from sklearn.model_selection import train_test_split
from collections import Counter
import pickle

In [39]:
x_list = []
y_list = []
targetlist_dict = {}

ct = 0
for brand in os.listdir('../phishpedia/expand_targetlist/'):
    for x in os.listdir(os.path.join('../phishpedia/expand_targetlist/', brand)):
        if x.endswith('.png') or x.endswith('.jpeg') or x.endswith('.JPG') or x.endswith('.jpg'):
            if not x.startswith('homepage') and not x.startswith('loginpage'):
                x_list.append(os.path.join(brand, x))
                y_list.append(ct)


    targetlist_dict[brand] = ct
    ct += 1

In [41]:
x_train, x_test, y_train, y_test = train_test_split(x_list, y_list, test_size=0.2, random_state=42,
                                                    stratify=y_list)

In [47]:
with open('train_targets.txt', 'a+') as f:
    for x in x_train:
        f.write(x+'\n')

In [48]:
with open('test_targets.txt', 'a+') as f:
    for x in x_test:
        f.write(x+'\n')

In [49]:
with open('target_dict.json', 'wb') as handle:
    pickle.dump(targetlist_dict, handle)

In [51]:
import torch.utils.data as data
from PIL import Image, ImageOps
import pickle
import numpy as np


class GetLoader(data.Dataset):
    def __init__(self, data_root, data_list, label_dict, transform=None, grayscale=False):
        
        self.transform = transform
        self.data_root = data_root
        self.grayscale = grayscale
        data_list = [x.strip('\n') for x in open(data_list).readlines()]

        with open(label_dict, 'rb') as handle:
            self.label_dict = pickle.load(handle)

        self.classes = list(self.label_dict.keys())

        self.n_data = len(data_list)

        self.img_paths = []
        self.labels = []

        for data in data_list:
            image_path = data
            label = image_path.split('/')[0]
            self.img_paths.append(image_path)
            self.labels.append(label)

    def __getitem__(self, item):

        img_path, label= self.img_paths[item], self.labels[item]
        img_path_full = os.path.join(self.data_root, img_path)
        if self.grayscale:
            img = Image.open(img_path_full).convert('L').convert('RGB')
        else:
            img = Image.open(img_path_full).convert('RGB')

        img = ImageOps.expand(img, (
            (max(img.size) - img.size[0]) // 2, (max(img.size) - img.size[1]) // 2,
            (max(img.size) - img.size[0]) // 2, (max(img.size) - img.size[1]) // 2), fill=(255, 255, 255))

        # label = np.array(label,dtype='float32')
        label = self.label_dict[label]
        if self.transform is not None:
            img = self.transform(img)

        return img, label

    def __len__(self):
        return self.n_data

In [52]:
train_set = GetLoader(data_root='../phishpedia/expand_targetlist',
                      data_list='train_targets.txt',
                      label_dict='target_dict.json',
                      transform=None)

## Convert RGBA into RGB

In [77]:
from PIL import Image
import cv2

In [102]:

for brand in os.listdir('../phishpedia/expand_targetlist/'):
    for x in os.listdir(os.path.join('../phishpedia/expand_targetlist/', brand)):
        if x.endswith('.png') or x.endswith('.jpeg') or x.endswith('.JPG') or x.endswith('.jpg'):
            if not x.startswith('homepage') and not x.startswith('loginpage'):
                img = cv2.imread(os.path.join('../phishpedia/expand_targetlist/', brand, x))
                if img.shape[-1] == 4:
                    print(img.shape)
                    img_new = img
    

In [105]:
for x in [x.strip() for x in open('test_targets.txt').readlines()]:
    if not os.path.exists(os.path.join('../phishpedia/expand_targetlist/', x)):
        print(x)

anadolubank/anadolubank-logo-E042456745-seeklogo.com.png
Delta Air Lines/18.png
